In [ ]:
import tensorflow as tf
from lightfm.datasets.movielens import fetch_movielens
import numpy as np


In [381]:
data_training['train'].toarray()[:100][0,1]


3

In [ ]:
def reshapeData(data):
    trainModified=a=b=c=d=np.array([])
    rows=cols=ratings=[]
    for row_index in range(0,data.shape[0]):
        for column_index in range(0,data.shape[1]) :
            rows.append(row_index)
            cols.append(column_index)
            ratings.append(data[row_index,column_index])
            print(str(ratings))
        
    a=np.array(rows)
    b=np.array(cols)
    c=np.array(ratings)
    return a,b,c

data_training=fetch_movielens()
a,b,c=reshapeData(data_training['train'].toarray()[:100,])
max_iter=100


In [342]:
def main():
    lda = 10.0
    rank = 5
    num_users=train.shape[0]
    num_items=train.shape[1]
    global_mean = np.mean(train[:,2])
    W, H, reg = create_factors_without_biases(num_users, num_items, rank, lda)
   
    tr, val, finalw, finalh = mf(train, test, W, H, reg, global_mean, max_iter, 1.0, True)
    print("Final training RMSE %s" % (tr))
    print("Final validation RMSE %s" % (val))

    np.save("final_w", finalw)
    np.save("final_h", finalh)

def extract_rating_info(ratings):
    rating_values = np.array(ratings[:,2], dtype=np.float32)
    user_indices = ratings[:,0]
    item_indices = ratings[:,1]
    num_ratings = len(item_indices)
    return rating_values, num_ratings, user_indices, item_indices

def create_factors_without_biases(num_users, num_items, rank, lda):
    # Initialize the matrix factors from random normals with mean 0. W will
    # represent users and H will represent items.
    W = tf.Variable(tf.truncated_normal([num_users, rank], stddev=0.02, mean=0), name="users")
    H = tf.Variable(tf.truncated_normal([rank, num_items], stddev=0.02, mean=0), name="items")
    regularizer = tf.multiply(tf.add(tf.reduce_sum(tf.square(W)), tf.reduce_sum(tf.square(H))), lda, name="regularize")
    return W, H, regularizer


In [275]:
def mf(ratings_train, ratings_val, W, H, regularizer, mean_rating, max_iter, lr = 0.01, decay_lr = False, log_summaries = True):
    # Extract info from training and validation data
    rating_values_tr, num_ratings_tr, user_indices_tr, item_indices_tr = extract_rating_info(ratings_train)
    rating_values_val, num_ratings_val, user_indices_val, item_indices_val = extract_rating_info(ratings_val)

    # Multiply the factors to get our result as a dense matrix
    result = tf.matmul(W, H)

    # Now we just want the values represented by the pairs of user and item
    # indices for which we had known ratings.
    result_values_tr = tf.gather(tf.reshape(result, [-1]), user_indices_tr * tf.shape(result)[1] + item_indices_tr, name="extract_training_ratings")
    result_values_val = tf.gather(tf.reshape(result, [-1]), user_indices_val * tf.shape(result)[1] + item_indices_val, name="extract_validation_ratings")

    # Calculate the difference between the predicted ratings and the actual
    # ratings. The predicted ratings are the values obtained form the matrix
    # multiplication with the mean rating added on.
    diff_op = tf.subtract(tf.add(result_values_tr, mean_rating, name="add_mean"), rating_values_tr, name="raw_training_error")
    diff_op_val = tf.subtract(tf.add(result_values_val, mean_rating, name="add_mean_val"), rating_values_val, name="raw_validation_error")

    with tf.name_scope("training_cost") as scope:
        base_cost = tf.reduce_sum(tf.square(diff_op, name="squared_difference"), name="sum_squared_error")

        cost = tf.div(tf.add(base_cost, regularizer), num_ratings_tr * 2, name="average_error")

    with tf.name_scope("validation_cost") as scope:
        cost_val = tf.div(tf.reduce_sum(tf.square(diff_op_val, name="squared_difference_val"), name="sum_squared_error_val"), num_ratings_val * 2, name="average_error")

    with tf.name_scope("train") as scope:
        if decay_lr:
            # Use an exponentially decaying learning rate.
            global_step = tf.Variable(0, trainable=False)
            learning_rate = tf.train.exponential_decay(lr, global_step, 10000, 0.96, staircase=True)
            optimizer = tf.train.GradientDescentOptimizer(learning_rate)
            # Passing global_step to minimize() will increment it at each step 
            # so that the learning rate will be decayed at the specified 
            # intervals.
            train_step = optimizer.minimize(cost, global_step=global_step)
        else:
            optimizer = tf.train.GradientDescentOptimizer(lr)
            train_step = optimizer.minimize(cost)

    with tf.name_scope("training_rmse") as scope:
        rmse_tr = tf.sqrt(tf.reduce_sum(tf.square(diff_op)) / num_ratings_tr)

    with tf.name_scope("validation_rmse") as scope:
      # Validation set rmse:
      rmse_val = tf.sqrt(tf.reduce_sum(tf.square(diff_op_val)) / num_ratings_val)

    # Create a TensorFlow session and initialize variables.
    sess = tf.Session()
    sess.run(tf.initialize_all_variables())

    if log_summaries:
        # Make sure summaries get written to the logs.
        accuracy_val_summary = tf.summary.scalar("accuracy_val", rmse_val)
#         accuracy_tr_summary = tf.summary.scalar("accuracy_tr", accuracy_tr)
        summary_op = tf.summary.merge_all()
        writer = tf.summary.FileWriter("/summary", sess.graph_def)
    # Keep track of cost difference.
    last_cost = 0
    diff = 1
    # Run the graph and see how we're doing on every 1000th iteration.
    for i in range(max_iter*100):
        if i > 0:
            if diff < 0.000001:
                print("Converged at iteration %s" % (i))
                break;
            if log_summaries:
                res = sess.run([rmse_tr, rmse_val, cost, summary_op])
                summary_str = res[3]
                writer.add_summary(summary_str, i)
            else:
                res = sess.run([rmse_tr, rmse_val, cost])
            acc_tr = res[0]
            acc_val = res[1]
            cost_ev = res[2]
            print("Training RMSE at step %s: %s" % (i, acc_tr))
            print("Validation RMSE at step %s: %s" % (i, acc_val))
            diff = abs(cost_ev - last_cost)
            last_cost = cost_ev
        else:
            sess.run(train_step)
            
    finalTrain = rmse_tr.eval(session=sess)
    finalVal = rmse_val.eval(session=sess)
    finalW = W.eval(session=sess)
    finalH = H.eval(session=sess)
#     print("final W",finalW)
#     print("final H",finalH)
    sorted_movies=tf.matmul(finalW,finalH).eval(session=sess)[0].argsort()
    print(finalW.shape)
    sess.close()
    return finalTrain, finalVal, finalW, finalH

In [276]:


main()

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Training RMSE at step 1: 470.281
Validation RMSE at step 1: 470.303
Training RMSE at step 2: 470.281
Validation RMSE at step 2: 470.303
Converged at iteration 3
(1586126, 5)
Final training RMSE 470.281
Final validation RMSE 470.303
